# Despartire in atomi

In [211]:
nume_fisier = "cerc.pas"

In [212]:
from pprint import pprint

with open(nume_fisier, 'r', encoding='utf-8') as f:
	cod = f.read()
	
atomi = cod.split()
#atomi = set(atomi)

pprint(atomi)

['program',
 'cerc',
 ';',
 'uses',
 'crt',
 ';',
 'var',
 'raza',
 ':',
 'real',
 ';',
 'perimetru',
 ':',
 'real',
 ';',
 'aria',
 ':',
 'real',
 ';',
 'detest',
 ':',
 'string',
 ';',
 'begin',
 '$',
 'clrscr',
 ';',
 'readln',
 '(',
 'raza',
 ')',
 ';$',
 'detest',
 ':=',
 "'SirDeCaractere'",
 ';',
 'writeln',
 '(',
 'detest',
 ')',
 '$',
 ';',
 'perimetru',
 ':=',
 '2',
 '*',
 '3.14',
 '*',
 'raza',
 ';',
 'aria',
 ':=',
 '3.14',
 '*',
 'raza',
 '*',
 'raza',
 ';',
 'writeln',
 '(',
 'perimetru',
 ')',
 ';',
 'writeln',
 '(',
 'aria',
 ')',
 ';',
 'end.']


In [213]:
def getKeywords(filename: str):
	keywords = {}
	with open(filename, 'r', encoding='utf-8') as f:
		lines = f.readlines()
		for line in lines:
			line = line.replace('\n', '')
			key, val = line.rsplit(':', 1)
			keywords[key] = val
	return keywords

In [214]:
class HashMap:
	def __init__(self, capacity=10):
		self.capacity = capacity  # Initial size of the hash table
		self.map = [None] * capacity  # Array to store key-value pairs

	def hash_function(self, key):
		hash_value = sum(ord(char) for char in str(key)) % self.capacity  # Sum ASCII values of key characters
		return hash_value

	def insert(self, key):
		hash_value = self.hash_function(key)
		index = hash_value

		# Handle collisions
		while self.map[index] is not None:
			index = (index + 1) % self.capacity

		self.map[index] = key

	def get_index(self, key):
		hash_value = self.hash_function(key)
		index = hash_value

		# Handle collisions
		while self.map[index] is not None:
			if self.map[index] == key:
				return index
			index = (index + 1) % self.capacity

		return None  # Key not found

	def get_by_index(self, index):
		return self.map[index]

	def print_map(self):
		for i in range(self.capacity):
			print(f"{i} : {self.map[i]}")
   
	def write_to_file(self):
		with open("ts.txt", "w") as f:
			for i in range(self.capacity):
				f.write(f"{i} : {self.map[i]} \n")

In [215]:
import re

def getTS(cod: str, keywords : dict):
	pattern_variables = r'\b(\w+)\s*:\s*(real|integer|string)\s*;'

	variables = re.findall(pattern_variables, cod)
	variables = [var[0] for var in variables]

	pattern_program = r'\bprogram\s+(\w+)\s*;'
	program = re.findall(pattern_program, cod)
	variables = variables + program

	for var in variables:
		if var in keywords.keys():
			variables.remove(var)

	ts = HashMap()
	for var in variables:
		ts.insert(var)

	pattern_constants = r'(\+|-|=|>|:=|<>|\*|/|\+=|-=|/=|%=)\s*(\d+\.*\d*)'
	pattern_strings = r"""(=|:=)\s*'([^"]*)'"""

	constants = re.findall(pattern_constants, cod)
	constants = [c[1] for c in constants]
	strings = re.findall(pattern_strings, cod)
	strings = [s[1] for s in strings]

	constants = constants + strings

	for c in constants:
		if c in keywords.keys():
			constants.remove(c)
		elif c in variables:
			constants.remove(c)

	constants = set(constants)

	for c in constants:
		ts.insert(c)

	return ts

In [216]:
from typing import List, Tuple

def getFIP(cod: str, keywords: dict, ts: HashMap):
    fip: List[Tuple[str, str]] = []
    exceptions = []
    
    line_nr = 0
    lines = cod.split("\n")
    for line in lines:
        line_nr += 1
        words = line.split()
        for word in words:
            clean_word = word.replace("'", '').replace('"', '')
            
            # Process and add to fip list with duplicates allowed
            if word in keywords.keys():
                fip.append((word, keywords[word]))
            elif ts.get_index(clean_word) is not None:
                if word[0].isalpha():
                    fip.append((word, str(keywords['ID']) + ' | ' + str(ts.get_index(clean_word))))
                else:
                    fip.append((word, str(keywords['CONST']) + ' | ' + str(ts.get_index(clean_word))))
            else:
                exceptions.append(f"The word '{word}' is neither a keyword, variable, nor a constant. Line {line_nr}")
    
    exception_message = None if not exceptions else "\n".join(exceptions)
    return fip, exception_message


# Main

In [217]:
keywords = getKeywords('keywords.csv')

cod = ""
with open('cerc.pas', 'r', encoding='utf-8') as f:
	cod = f.read()

ts = getTS(cod, keywords)

# Print the TS table
print('TS')
print('--------------------')
ts.print_map()
ts.write_to_file()
print('--------------------')

TS
--------------------
0 : raza
1 : detest
2 : 2
3 : aria
4 : cerc
5 : None
6 : None
7 : SirDeCaractere
8 : 3.14
9 : perimetru
--------------------


In [220]:
fip, exception2 = getFIP(cod, keywords, ts)

In [227]:
for el in fip:
    print(el)
    
with open("fip.txt", "w") as f:
	for el in fip:
		f.write(el[0] + " " + el[1] + "\n")
	if exception2 != None:
		f.write(exception2)

('program', '16')
('cerc', '0 | 4')
(';', '11')
('uses', '18')
('crt', '14')
(';', '11')
('var', '19')
('raza', '0 | 0')
(':', '41')
('real', '4')
(';', '11')
('perimetru', '0 | 9')
(':', '41')
('real', '4')
(';', '11')
('aria', '0 | 3')
(':', '41')
('real', '4')
(';', '11')
('detest', '0 | 1')
(':', '41')
('string', '5')
(';', '11')
('begin', '12')
('clrscr', '13')
(';', '11')
('readln', '17')
('(', '6')
('raza', '0 | 0')
(')', '7')
('detest', '0 | 1')
(':=', '42')
("'SirDeCaractere'", '1 | 7')
(';', '11')
('writeln', '20')
('(', '6')
('detest', '0 | 1')
(')', '7')
(';', '11')
('perimetru', '0 | 9')
(':=', '42')
('2', '1 | 2')
('*', '8')
('3.14', '1 | 8')
('*', '8')
('raza', '0 | 0')
(';', '11')
('aria', '0 | 3')
(':=', '42')
('3.14', '1 | 8')
('*', '8')
('raza', '0 | 0')
('*', '8')
('raza', '0 | 0')
(';', '11')
('writeln', '20')
('(', '6')
('perimetru', '0 | 9')
(')', '7')
(';', '11')
('writeln', '20')
('(', '6')
('aria', '0 | 3')
(')', '7')
(';', '11')
('end.', '15')
